## 예제
1. csv 폴더에서 4개의 파일을 로드 (tran1, tran2, tran_d_1, trand_d_2)
2. tran_1과 tran_2는 단순한 행 추가 결합 (유니언 결합)
3. tran_d_1과 tran_d_2도 유니언 결합
4. 2번, 3번 과정에서 나온 데이터프레임을 특정 조건에 맞춰서 2번 데이터프레임 기준으로 열 추가 결합(조인 결합)
5. csv 폴더에 있는 2개의 파일 로드 (custom_master, item_master)
6. 4번 과정에서 나온 데이터프레임과 custom_master 조인 결합
7. 6번 과정에서 나온 데이터프레임과 item_master 조인 결합
8. 결합이 된 데이터프레임에서 quantity 컬럼과 item_price 컬럼의 데이터를 가지고 total_price라는 파생변수를 생성(total_price = item_price * quantity)

In [ ]:
import pandas as pd

In [ ]:
tran1 = pd.read_csv("../csv/tran_1.csv")
tran2 = pd.read_csv("../csv/tran_2.csv")
trand1 = pd.read_csv("../csv/tran_d_1.csv")
trand2 = pd.read_csv("../csv/tran_d_2.csv")

In [ ]:
# tran1, tran2 유니언결합
tran = pd.concat([tran1, tran2], axis=0, ignore_index=True)

In [ ]:
# trand1, trand2 유니언 결합
trand = pd.concat([trand1, trand2], axis= 0, ignore_index=True)

In [ ]:
tran.head(1)

In [ ]:
trand.head(1)

In [ ]:
# tran, trand 를 조인 결합 (조건  : transaction_id, how : tran)
tran =  pd.merge(tran, trand, on='transaction_id', how='left')

In [ ]:
# 2개 파일을 로드 
customer_master = pd.read_csv("../csv/customer_master.csv")
item_master = pd.read_csv("../csv/item_master.csv")

In [ ]:
# tran, cutomer_master 조인 결합 (조건 : customer_id, how : inner, left)
total_df = pd.merge(tran, customer_master, on = 'customer_id', how='inner')

In [ ]:
# total_df, item_master 조인결합 (조건  : item_id, how : inner)
total_df = pd.merge(total_df, item_master, on='item_id', how = 'inner')

In [ ]:
# 새로운 파생변수 생성 total_price
total_df['total_price'] = total_df['item_price'] * total_df['quantity']

In [ ]:
total_df.head(1)

In [ ]:
# 거래의 금액이 가장 좋은 아이템은 무엇인가?
group_data = total_df[['item_name', 'total_price']].groupby('item_name').sum()

In [ ]:
# 내림차순 정렬 
group_data.sort_values('total_price', ascending=False).index[0]

In [ ]:
total_df['gender']

In [ ]:
origin_data = total_df.copy()

In [ ]:
# loc를 이용하여 m, f 를 변경
# m->남자, f->여자
total_df.loc[total_df['gender']  == "M", 'gender'] = '남자'
total_df.loc[total_df['gender']  == "F", 'gender'] = '여자'

In [ ]:
total_df['gender'].value_counts()

In [ ]:
total_df = origin_data.copy()

In [ ]:
total_df['gender'].value_counts()

In [ ]:
# apply() 사용하여 남자, 여자 변경

def change(x):
    # x 에는 M아니면 F
    if x == "F":
        result = "여자"
    else:
        result = "남자"
    return result

total_df['gender'].apply(change)

In [ ]:
total_df['gender'].apply(lambda x : "남자" if (x == 'M') else 
                            ("여자" if(x == "F") else "무응답")
                        )

In [ ]:
# payment_date 컬럼을 시계열 데이터 변경
# to_datetime()
total_df.head(1)

In [ ]:
total_df['payment_date'] = pd.to_datetime(total_df['payment_date'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
total_df.info()

In [ ]:
# 시계열 데이터에서 월-년 추출해서 새로운 파생변수 payment_month 생성
total_df['payment_month'] = total_df['payment_date'].dt.strftime('%m-%Y')

In [ ]:
total_df.head(1)

In [68]:
# 월별 거래량을 확인
group_data = total_df[['payment_month', 'total_price']].groupby('payment_month').agg(['mean', 'sum'])

In [69]:
group_data

total_price           
                        mean        sum
payment_month                          
02-2019        137592.964824  136905000
03-2019        135154.382470  135695000
04-2019        136106.106106  135970000
05-2019        133551.307847  132750000
06-2019        135747.800587  138870000
07-2019        138377.609108  145850000

In [63]:
pivot_data = pd.pivot_table(
    total_df, 
    index = 'payment_month', 
    aggfunc = ['sum', 'mean', 'min', 'max', 'std'], 
    values = 'total_price'
)